<a href="https://colab.research.google.com/github/Markkolas/TP2/blob/jorge-test/Pruebas_API/Deploying_ML_model_as_public_API_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the dependencies

In [ ]:
# @title Mount & Clone

# Imports
import requests

# @markdown Mount GDrive
Gdrive = False  # @param {type: 'boolean'}
global path

# @markdown Clone Repository (Github)
repositoryUrl = "" # @param {type: 'string'}

class MountClone:
    def __init__(
            self,
            Gdrive=False,
            repositoryUrl="",
            verbose=False):
        
        if Gdrive:
            self.mount_drive()
        if repositoryUrl:
            self.clone_repo(repositoryUrl, verbose)

    def __str__(self):
        try:
            return self.path
        except NameError:
            return ''
        
    def mount_drive(self):
        from google.colab import drive
        drive.mount('/content/drive')
        self.path = "/content/drive"

    def clone_repo(self, repositoryUrl, verbose=False):
        response = requests.get(repositoryUrl)
        if response.status_code == 200:
            print("✔️ Public repository")
            ! cd /content
            ! git clone $repositoryUrl
            folder = repositoryUrl.split("/")[-1]
            self.path = f"/content/{folder}"
        else:
            print("❌ Not a public Repository")

if __name__ == "__main__":
    path = MountClone(Gdrive=Gdrive, repositoryUrl=repositoryUrl)

In [15]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━

In [13]:
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import torch
import numpy as np

In [5]:
app = FastAPI()

In [6]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [7]:
# class model_input(BaseModel):
    
#     Pregnancies : int
#     Glucose : int
#     BloodPressure : int
#     SkinThickness : int
#     Insulin : int
#     BMI : float
#     DiabetesPedigreeFunction : float
#     Age : int

class image(BaseModel):
  img : str

In [26]:
# loading the saved model
# diabetes_model = pickle.load(open('diabetes_model.sav', 'rb'))
modelo = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/FiveClasses.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-5-11 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


Model summary: 157 layers, 7023610 parameters, 0 gradients
Adding AutoShape... 


In [22]:
@app.post('/clasification')
async def create_upload_file(file: UploadFile = File(...)):
    results = modelo(file)
    return results
    

    
    # input_data = input_parameters.json()
    # input_dictionary = json.loads(input_data)
    
    # preg = input_dictionary['Pregnancies']
    # glu = input_dictionary['Glucose']
    # bp = input_dictionary['BloodPressure']
    # skin = input_dictionary['SkinThickness']
    # insulin = input_dictionary['Insulin']
    # bmi = input_dictionary['BMI']
    # dpf = input_dictionary['DiabetesPedigreeFunction']
    # age = input_dictionary['Age']
    
    
    # input_list = [preg, glu, bp, skin, insulin, bmi, dpf, age]
    
    # prediction = diabetes_model.predict([input_list])
    
    # if (prediction[0] == 0):
    #     return 'The person is not diabetic'
    # else:
    #     return 'The person is diabetic'

In [31]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import StreamingResponse
from io import BytesIO

app = FastAPI()

@app.post("/classification")
async def classify_file(file: UploadFile = File(...)):
    # Process the uploaded file here
    file_content = await file.read()
    results = modelo(file_content)

    # Create a response that returns the results as a file
    file_like = BytesIO(results)
    return StreamingResponse(file_like, media_type='application/octet-stream', headers={'Content-Disposition': 'attachment; filename=results.bin'})


In [38]:
import io
import requests
from PIL import Image
import torch
import torchvision
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import StreamingResponse

app = FastAPI()

# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Ruta para la detección de objetos
@app.post("/deteccion-objetos")
async def detect_objects(file: UploadFile = File(...)):
    # Leer la imagen cargada
    contents = await file.read()
    image = Image.open(io.BytesIO(contents)).convert('RGB')

    # Ejecutar la detección de objetos
    results = model(image)

    # Guardar los resultados en un archivo
    file_like = io.BytesIO()
    results.print()  # Imprimir los resultados en la consola para depuración
    results.save(file_like, format='json')

    # Devolver la respuesta como un archivo
    file_like.seek(0)
    return StreamingResponse(file_like, media_type='application/json', headers={'Content-Disposition': 'attachment; filename=results.json'})


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-11 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [40]:
from fastapi import FastAPI, File, UploadFile
from PIL import Image
from fastapi.responses import HTMLResponse

app = FastAPI()

@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile = File(...)):
    with Image.open(file.file) as image:
        image.thumbnail((640, 640))
        image.save(file.filename)
    return HTMLResponse(content=f"<img src='{file.filename}'/>", status_code=200)


In [43]:
from fastapi import FastAPI
app = FastAPI()
# Python main entrypoint
if __name__ == '__main__':
    import uvicorn
    # APP/app_str - The ASGI application to run, in the format "<module>:<attribute>"
    app_str = 'server_minimal:app'
    uvicorn.run(app_str, host='localhost', port=8000, reload=True, workers=1)

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [405] using StatReload
INFO:     Stopping reloader process [405]


In [46]:
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel
from typing import List, Optional

def results_to_json(results, model):
    ''' Converts yolo model output to json (list of list of dicts)'''
    return [
                [
                    {
                    "class": int(pred[5]),
                    "class_name": model.model.names[int(pred[5])],
                    "bbox": [int(x) for x in pred[:4].tolist()], #convert bbox results to int from float
                    "confidence": float(pred[4]),
                    }
                for pred in result
                ]
            for result in results.xyxy
            ]

@app.post("/yolo/")
async def process_home_form(file: UploadFile = File(...), model_name: str = Form(...)):
  
      model = torch.hub.load('ultralytics/yolov5', model_name, pretrained=True, force_reload = False)
      #This is how you decode + process image with PIL
      results = model(Image.open(BytesIO(await file.read())))
      json_results = results_to_json(results,model)
      return json_results

In [47]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://0699-34-125-231-164.ngrok.io


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [405]


In [48]:
!python server.py

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [32315] using StatReload
INFO:     Started server process [32330]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [32330]
INFO:     Stopping reloader process [32315]
